# Search and Access NISAR GCOV Data with `asf_search`

:::{warning} Current state of NISAR data access
At the time of writing (2025-12-04), there are no NISAR GCOV data in production. None of the searches in this notebook will produce results until production NISAR L2 GCOV data are released.
:::

[`asf_search`](https://github.com/asfadmin/Discovery-asf_search) is an open source Python package for searching and downloading SAR data from {abbr}`ASF (Alaska Satellite Facility)`. This notebook demonstrates how to search and download NISAR GCOV data with `asf_search`.

<hr>

## Overview

1. [Prerequisites](prereqs)
1. [`asf_search.geo_search()` Example](geo)
1. [Generic `asf_search.search()` example](search)
1. [Stream the data](stream)
1. [Summary](summary)
1. [Resources and references](resources)

<hr>

(prereqs)=
## 1. Prerequisites
| Prerequisite | Importance | Notes |
| --- | --- | --- |
| [The software environment for this cookbook must be installed](https://github.com/ASFOpenSARlab/NISAR_GCOV_Cookbook/blob/main/notebooks/create_software_environment.ipynb) | Necessary | |
| [Familiarity with `asf_search`](https://docs.asf.alaska.edu/asf_search/basics/) | helpful | |

- **Rough Notebook Time Estimate**: 3 minutes

<hr>

(geo)=
## 2. `asf_search.geo_search()` Example

### Perform a geographic search and fetch product URLs

In [ ]:
import asf_search as asf
from datetime import datetime

start_date = datetime(2025, 11, 20)
end_date = datetime(2025, 12, 1)
area_of_interest = "POLYGON((-104.826 32.4865,-100.0806 32.4865,-100.0806 38.134,-104.826 38.134,-104.826 32.4865))" # POINT or POLYGON as WKT (well-known-text)

response = asf.geo_search(
    intersectsWith=area_of_interest,
    platform=asf.PLATFORM.NISAR,
    start=start_date,
    end=end_date
)

hdf5_files = response.find_urls(extension='.h5', directAccess=False)
hdf5_files

### Filter the results

If your search results contain GCOV product files that you don't want, you can remove them with a pattern (regex) filter when calling `response.find_urls`

:::{tip} Reference the GCOV File Naming Convention
Refer to the File Naming Convention section (3.4) of the [NISAR L2 GCOV Product Specification](https://nisar.asf.earthdatacloud.nasa.gov/NISAR-SAMPLE-DATA/DOCS/NISAR_D-102274_RevE_NASA_SDS_Product_Specification_L2_GCOV_Nov8_2024_w-sigs.pdf) when creating your regex pattern.
:::

:::{tip} New to regex?
Here are some resources to get you started:
- [Regex Tutorial](https://www.geeksforgeeks.org/dsa/write-regular-expressions/)
- [Regex Sandbox](https://regex101.com/)
  
:::

In [ ]:
# Includes files that contain "DHDH" and are not preceded by "STATS"
pattern = r'^(?!.*STATS).*DHDH.*' 

hdf5_files = response.find_urls(extension='.h5', pattern=pattern, directAccess=False)
hdf5_files

### Download the data to a specified directory

In [ ]:
data_dir = "GCOV_data"

session = asf.ASFSession()
session.auth_with_creds(input('EDL Username'), getpass('EDL Password'))
asf.download_urls(hdf5_files, data_dir, session=session, processes=4)

<hr>

(search)=
## 3. Generic `asf_search.search()` example

:::{tip} View all possible search parameters 

There are many `ASFSearchOptions` parameters, in addition to those used in the search below. A complete list can be found [here](https://github.com/asfadmin/Discovery-asf_search/blob/3db7664fce8c0e745623ac28e923594182cc1451/asf_search/ASFSearchOptions/validator_map.py#L41-L101).
:::

In [ ]:
import os
import asf_search as asf
from getpass import getpass

session = asf.ASFSession()
session.auth_with_creds(input('EDL Username'), getpass('EDL Password'))

opts=asf.ASFSearchOptions(**{
    "maxResults": 250,
    "processingLevel": [
        "GCOV"
    ],
    "dataset": [
        "NISAR"
    ],
    "productionConfiguration": [
        "PR"
    ],
    'session': session,
})

response = asf.search(opts=opts)
response

### Retrieve the product URLs

:::{tip} Filter the results

See Section 1 for guidance on filtering the results when calling `response.find_urls`
:::

In [ ]:
hdf5_files = response.find_urls(extension='.h5', directAccess=False)
hdf5_files

### Download the data to a specified directory

In [ ]:
data_dir = "GCOV_data"

asf.download_urls(hdf5_files, data_dir, session=session, processes=4)

<hr>

(stream)=
## 4. Stream the data

NISAR data products can be very large. It may be helpful to access the data in-place, and steam only the subsets that you need.

:::{warning} Coming soon

We will build out the data streaming section when GCOV data are available.
:::

<hr>

(summary)=
## 5. Summary
You now have the tools and knowledge that you need to search and download data using the `asf_search` Python package.

<hr>

(resources)=
## 6. Resources and references
 - [asf_search](https://github.com/asfadmin/Discovery-asf_search)
 
**Author:** Alex Lewandowski